In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_2_Medoid_Cluster_2_pvalue_02.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [3]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))


In [4]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'


In [5]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp



In [6]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)



In [7]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:
    print(pair)
    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"


file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_2_Medoids_Cluster_2_pValue_02.txt', 'w')
file.write(pos_result)
file.close()



['AAP', 'ALK']
['AAP', 'AMD']
['AAP', 'CMG']
['AAP', 'EXPE']
['AAP', 'NEM']
['ABC', 'CNC']
['ABC', 'DLTR']
['ABC', 'EBAY']
['ABC', 'KMX']
['ABC', 'KSS']
['ADI', 'AGN']
['ADI', 'CHRW']
['ADI', 'CI']
['ADI', 'CNP']
['ADI', 'COG']
['ADI', 'COL']
['ADI', 'ETN']
['ADI', 'GPS']
['ADI', 'HBI']
['ADI', 'HUM']
['ADI', 'ILMN']
['ADI', 'IR']
['ADI', 'KEY']
['ADI', 'KMX']
['ADI', 'KSU']
['ADI', 'LH']
['ADI', 'NEM']
['ADI', 'NTAP']
['ADI', 'PRGO']
['ADI', 'RL']
['ADI', 'RRC']
['ADI', 'TXT']
['ADI', 'VMC']
['ADI', 'WU']
['ADI', 'YUM']
['ADM', 'CF']
['ADM', 'CMG']
['ADM', 'CMI']
['ADM', 'COG']
['ADM', 'COO']
['ADM', 'FFIV']
['ADM', 'HBI']
['ADM', 'HCA']
['ADM', 'ILMN']
['ADM', 'JWN']
['ADM', 'OKE']
['ADM', 'PRGO']
['ADM', 'PVH']
['ADM', 'PWR']
['ADM', 'RRC']
['ADM', 'STX']
['ADM', 'UHS']
['ADM', 'WDC']
['ADM', 'XEC']
['ADP', 'AEE']
['ADP', 'AEP']
['ADP', 'AES']
['ADP', 'AFL']
['ADP', 'AIV']
['ADP', 'ALGN']
['ADP', 'ALK']
['ADP', 'ALL']
['ADP', 'ANDV']
['ADP', 'ANSS']
['ADP', 'APC']
['ADP', 'APD']
['A

['AGN', 'RHT']
['AGN', 'RJF']
['AGN', 'ROK']
['AGN', 'SBAC']
['AGN', 'SEE']
['AGN', 'SNPS']
['AGN', 'STI']
['AGN', 'STX']
['AGN', 'STZ']
['AGN', 'TDG']
['AGN', 'TEL']
['AGN', 'TSN']
['AGN', 'TXT']
['AGN', 'UAL']
['AGN', 'VFC']
['AGN', 'VIAB']
['AGN', 'VLO']
['AGN', 'VMC']
['AGN', 'VNO']
['AGN', 'VRSK']
['AGN', 'WDC']
['AGN', 'WHR']
['AGN', 'WMB']
['AGN', 'WYNN']
['AGN', 'XEC']
['AGN', 'XRX']
['AGN', 'XYL']
['AGN', 'YUM']
['AGN', 'ZTS']
['AIG', 'AIV']
['AIG', 'AJG']
['AIG', 'ALL']
['AIG', 'AME']
['AIG', 'ANSS']
['AIG', 'ARE']
['AIG', 'ARNC']
['AIG', 'AVB']
['AIG', 'AVY']
['AIG', 'AWK']
['AIG', 'BBT']
['AIG', 'BEN']
['AIG', 'BFB']
['AIG', 'BLL']
['AIG', 'BXP']
['AIG', 'CA']
['AIG', 'CAG']
['AIG', 'CCI']
['AIG', 'CERN']
['AIG', 'CF']
['AIG', 'CHD']
['AIG', 'CHRW']
['AIG', 'CI']
['AIG', 'CINF']
['AIG', 'CLX']
['AIG', 'CMS']
['AIG', 'CNP']
['AIG', 'COF']
['AIG', 'COG']
['AIG', 'CPB']
['AIG', 'CRM']
['AIG', 'CTL']
['AIG', 'CTXS']
['AIG', 'D']
['AIG', 'DE']
['AIG', 'DG']
['AIG', 'DGX']
['AIG'

['ALB', 'LKQ']
['ALB', 'LLL']
['ALB', 'LNT']
['ALB', 'LRCX']
['ALB', 'LUV']
['ALB', 'MA']
['ALB', 'MAA']
['ALB', 'MAC']
['ALB', 'MAR']
['ALB', 'MAS']
['ALB', 'MAT']
['ALB', 'MCHP']
['ALB', 'MCK']
['ALB', 'MCO']
['ALB', 'MGM']
['ALB', 'MHK']
['ALB', 'MKC']
['ALB', 'MLM']
['ALB', 'MMC']
['ALB', 'MNST']
['ALB', 'MPC']
['ALB', 'MSI']
['ALB', 'MTB']
['ALB', 'MTD']
['ALB', 'NCLH']
['ALB', 'NDAQ']
['ALB', 'NFLX']
['ALB', 'NLSN']
['ALB', 'NSC']
['ALB', 'NTAP']
['ALB', 'NTRS']
['ALB', 'NWL']
['ALB', 'OMC']
['ALB', 'ORLY']
['ALB', 'PAYX']
['ALB', 'PBCT']
['ALB', 'PCG']
['ALB', 'PEG']
['ALB', 'PGR']
['ALB', 'PHM']
['ALB', 'PKG']
['ALB', 'PLD']
['ALB', 'PNW']
['ALB', 'PPG']
['ALB', 'PPL']
['ALB', 'PRGO']
['ALB', 'PRU']
['ALB', 'PSA']
['ALB', 'PVH']
['ALB', 'PWR']
['ALB', 'PX']
['ALB', 'RCL']
['ALB', 'RE']
['ALB', 'REG']
['ALB', 'REGN']
['ALB', 'RF']
['ALB', 'RHI']
['ALB', 'RHT']
['ALB', 'RJF']
['ALB', 'ROK']
['ALB', 'ROP']
['ALB', 'ROST']
['ALB', 'RSG']
['ALB', 'SBAC']
['ALB', 'SCG']
['ALB', 'SEE'

['ALXN', 'PCG']
['ALXN', 'PEG']
['ALXN', 'PGR']
['ALXN', 'PH']
['ALXN', 'PHM']
['ALXN', 'PKG']
['ALXN', 'PLD']
['ALXN', 'PNR']
['ALXN', 'PNW']
['ALXN', 'PPG']
['ALXN', 'PPL']
['ALXN', 'PRGO']
['ALXN', 'PRU']
['ALXN', 'PSA']
['ALXN', 'PVH']
['ALXN', 'PWR']
['ALXN', 'PX']
['ALXN', 'PXD']
['ALXN', 'RCL']
['ALXN', 'RE']
['ALXN', 'REG']
['ALXN', 'RF']
['ALXN', 'RJF']
['ALXN', 'RL']
['ALXN', 'ROK']
['ALXN', 'ROP']
['ALXN', 'ROST']
['ALXN', 'RRC']
['ALXN', 'RSG']
['ALXN', 'SBAC']
['ALXN', 'SCG']
['ALXN', 'SHW']
['ALXN', 'SJM']
['ALXN', 'SLG']
['ALXN', 'SNA']
['ALXN', 'SNPS']
['ALXN', 'SO']
['ALXN', 'SPG']
['ALXN', 'SRCL']
['ALXN', 'SRE']
['ALXN', 'STI']
['ALXN', 'STT']
['ALXN', 'STX']
['ALXN', 'STZ']
['ALXN', 'SWK']
['ALXN', 'SYMC']
['ALXN', 'SYY']
['ALXN', 'TDG']
['ALXN', 'TEL']
['ALXN', 'TGT']
['ALXN', 'TIF']
['ALXN', 'TMK']
['ALXN', 'TPR']
['ALXN', 'TROW']
['ALXN', 'TRV']
['ALXN', 'TSN']
['ALXN', 'TXT']
['ALXN', 'UAL']
['ALXN', 'UDR']
['ALXN', 'ULTA']
['ALXN', 'UNM']
['ALXN', 'V']
['ALXN',

['ANSS', 'WY']
['ANSS', 'XEL']
['ANSS', 'XYL']
['ANSS', 'ZION']
['AON', 'BEN']
['AON', 'CAH']
['AON', 'CF']
['AON', 'CI']
['AON', 'COG']
['AON', 'CRM']
['AON', 'CTSH']
['AON', 'CTXS']
['AON', 'DAL']
['AON', 'DE']
['AON', 'DG']
['AON', 'DISH']
['AON', 'DLR']
['AON', 'ETN']
['AON', 'HBAN']
['AON', 'HOG']
['AON', 'HUM']
['AON', 'ILMN']
['AON', 'INTU']
['AON', 'IR']
['AON', 'KEY']
['AON', 'LNC']
['AON', 'MHK']
['AON', 'PH']
['AON', 'PNR']
['AON', 'PPG']
['AON', 'PRGO']
['AON', 'ROK']
['AON', 'STZ']
['AON', 'TXT']
['AON', 'VLO']
['AON', 'YUM']
['AOS', 'CRM']
['AOS', 'EXC']
['AOS', 'MA']
['AOS', 'TSCO']
['APA', 'NBL']
['APD', 'CAG']
['APD', 'CERN']
['APD', 'CF']
['APD', 'CMI']
['APD', 'COF']
['APD', 'COG']
['APD', 'COO']
['APD', 'CTL']
['APD', 'DAL']
['APD', 'DG']
['APD', 'DHI']
['APD', 'DISH']
['APD', 'F']
['APD', 'FFIV']
['APD', 'FIS']
['APD', 'FLIR']
['APD', 'GPS']
['APD', 'HBAN']
['APD', 'HBI']
['APD', 'HOG']
['APD', 'HST']
['APD', 'HSY']
['APD', 'HUM']
['APD', 'IP']
['APD', 'IR']
['APD'

...

['K', 'ZION']
['KEY', 'LLL']
['KEY', 'MCHP']
['KEY', 'MCK']
['KEY', 'MCO']
['KEY', 'MET']
['KEY', 'MHK']
['KEY', 'MLM']
['KEY', 'MPC']
['KEY', 'NBL']
['KEY', 'NLSN']
['KEY', 'NOV']
['KEY', 'PCAR']
['KEY', 'PH']
['KEY', 'PNR']
['KEY', 'PNW']
['KEY', 'PPG']
['KEY', 'PRGO']
['KEY', 'PWR']
['KEY', 'RE']
['KEY', 'REGN']
['KEY', 'RHI']
['KEY', 'RHT']
['KEY', 'ROK']
['KEY', 'SCG']
['KEY', 'SEE']
['KEY', 'SNPS']
['KEY', 'SO']
['KEY', 'STZ']
['KEY', 'TMK']
['KEY', 'TXT']
['KEY', 'UHS']
['KEY', 'UNM']
['KEY', 'VLO']
['KEY', 'VTR']
['KEY', 'XYL']
['KEY', 'YUM']
['KIM', 'LKQ']
['KIM', 'LYB']
['KIM', 'MAC']
['KIM', 'MET']
['KIM', 'NFLX']
['KIM', 'NUE']
['KIM', 'PCG']
['KIM', 'PFG']
['KIM', 'PLD']
['KIM', 'PRGO']
['KIM', 'PRU']
['KIM', 'PWR']
['KIM', 'REG']
['KIM', 'ROK']
['KIM', 'SCG']
['KIM', 'SO']
['KIM', 'SPG']
['KIM', 'TRIP']
['KIM', 'UAL']
['KIM', 'UHS']
['KIM', 'WEC']
['KIM', 'XEC']
['KIM', 'XEL']
['KIM', 'YUM']
['KIM', 'ZION']
['KMB', 'KMX']
['KMB', 'MAT']
['KMB', 'MPC']
['KMB', 'NTAP']
['KM

['MAS', 'TDG']
['MAS', 'TEL']
['MAS', 'TGT']
['MAS', 'TIF']
['MAS', 'TMK']
['MAS', 'TPR']
['MAS', 'TRIP']
['MAS', 'TROW']
['MAS', 'TRV']
['MAS', 'TSCO']
['MAS', 'TSN']
['MAS', 'TSS']
['MAS', 'TXT']
['MAS', 'UAL']
['MAS', 'UDR']
['MAS', 'UHS']
['MAS', 'UNM']
['MAS', 'V']
['MAS', 'VFC']
['MAS', 'VIAB']
['MAS', 'VLO']
['MAS', 'VMC']
['MAS', 'VNO']
['MAS', 'VRSK']
['MAS', 'VRSN']
['MAS', 'VRTX']
['MAS', 'VTR']
['MAS', 'WAT']
['MAS', 'WDC']
['MAS', 'WEC']
['MAS', 'WHR']
['MAS', 'WM']
['MAS', 'WMB']
['MAS', 'WU']
['MAS', 'WY']
['MAS', 'WYNN']
['MAS', 'XEC']
['MAS', 'XEL']
['MAS', 'XL']
['MAS', 'XLNX']
['MAS', 'XRAY']
['MAS', 'XRX']
['MAS', 'XYL']
['MAS', 'YUM']
['MAS', 'ZION']
['MAS', 'ZTS']
['MAT', 'MNST']
['MAT', 'ORLY']
['MAT', 'REGN']
['MAT', 'UAA']
['MAT', 'VRTX']
['MCHP', 'NEM']
['MCHP', 'NLSN']
['MCHP', 'OMC']
['MCHP', 'PGR']
['MCHP', 'PNR']
['MCHP', 'PVH']
['MCHP', 'PX']
['MCHP', 'RL']
['MCHP', 'RRC']
['MCHP', 'SLG']
['MCHP', 'STI']
['MCHP', 'STZ']
['MCHP', 'TEL']
['MCHP', 'TPR']
['M

['MTB', 'UHS']
['MTB', 'UNM']
['MTB', 'VLO']
['MTB', 'VMC']
['MTB', 'VNO']
['MTB', 'VRSK']
['MTB', 'VRSN']
['MTB', 'VRTX']
['MTB', 'VTR']
['MTB', 'WEC']
['MTB', 'WHR']
['MTB', 'WM']
['MTB', 'WY']
['MTB', 'XEL']
['MTB', 'XL']
['MTB', 'XRAY']
['MTB', 'XYL']
['MTB', 'YUM']
['MTB', 'ZION']
['MTB', 'ZTS']
['MTD', 'OKE']
['MTD', 'SYMC']
['MTD', 'VRTX']
['MYL', 'NI']
['NBL', 'PXD']
['NBL', 'TRIP']
['NBL', 'URI']
['NCLH', 'NEM']
['NCLH', 'NFX']
['NCLH', 'NOV']
['NCLH', 'NSC']
['NCLH', 'NUE']
['NCLH', 'OKE']
['NCLH', 'ORLY']
['NCLH', 'PHM']
['NCLH', 'PRGO']
['NCLH', 'PWR']
['NCLH', 'PXD']
['NCLH', 'REGN']
['NCLH', 'RL']
['NCLH', 'ROST']
['NCLH', 'SEE']
['NCLH', 'SNPS']
['NCLH', 'SRCL']
['NCLH', 'TGT']
['NCLH', 'TRIP']
['NCLH', 'TSCO']
['NCLH', 'URI']
['NCLH', 'V']
['NCLH', 'VIAB']
['NCLH', 'VRTX']
['NCLH', 'WHR']
['NCLH', 'XEC']
['NCLH', 'ZTS']
['NDAQ', 'NWL']
['NDAQ', 'PWR']
['NDAQ', 'RCL']
['NDAQ', 'UAL']
['NDAQ', 'WY']
['NEM', 'NFX']
['NEM', 'NI']
['NEM', 'RCL']
['NFX', 'TRIP']
['NLSN', 'STZ

['PRGO', 'STX']
['PRU', 'PSA']
['PRU', 'PWR']
['PRU', 'PX']
['PRU', 'RE']
['PRU', 'REG']
['PRU', 'RL']
['PRU', 'ROK']
['PRU', 'SCG']
['PRU', 'SLG']
['PRU', 'SO']
['PRU', 'SPG']
['PRU', 'SRE']
['PRU', 'SYMC']
['PRU', 'SYY']
['PRU', 'TDG']
['PRU', 'TGT']
['PRU', 'TIF']
['PRU', 'TPR']
['PRU', 'UDR']
['PRU', 'UHS']
['PRU', 'VMC']
['PRU', 'VTR']
['PRU', 'WEC']
['PRU', 'XEL']
['PSA', 'PVH']
['PSA', 'PWR']
['PSA', 'ROP']
['PSA', 'STX']
['PSA', 'SYY']
['PSA', 'TIF']
['PSA', 'UHS']
['PSA', 'WDC']
['PSA', 'YUM']
['PSA', 'ZION']
['PWR', 'ROK']
['PWR', 'RRC']
['PWR', 'SCG']
['PWR', 'TRIP']
['PWR', 'TSS']
['PWR', 'ULTA']
['PWR', 'VRSN']
['PWR', 'XRAY']
['PWR', 'YUM']
['PX', 'RF']
['PX', 'RHI']
['PX', 'SRE']
['PX', 'STI']
['PX', 'TROW']
['PX', 'UNM']
['PXD', 'TRIP']
['RCL', 'RE']
['RCL', 'REG']
['RCL', 'REGN']
['RCL', 'RF']
['RCL', 'RHI']
['RCL', 'RHT']
['RCL', 'RJF']
['RCL', 'RL']
['RCL', 'ROK']
['RCL', 'ROP']
['RCL', 'ROST']
['RCL', 'RSG']
['RCL', 'SBAC']
['RCL', 'SCG']
['RCL', 'SEE']
['RCL', 'SHW

['STT', 'TAP']
['STT', 'TGT']
['STT', 'TIF']
['STT', 'TSN']
['STT', 'UAA']
['STT', 'UDR']
['STT', 'UHS']
['STT', 'VLO']
['STT', 'VRSK']
['STT', 'VRTX']
['STT', 'VTR']
['STT', 'WAT']
['STT', 'WEC']
['STT', 'WHR']
['STT', 'WU']
['STT', 'WY']
['STT', 'XEL']
['STT', 'ZTS']
['STX', 'WDC']
['STX', 'WHR']
['STX', 'WMB']
['STX', 'WYNN']
['STZ', 'SYMC']
['STZ', 'TEL']
['STZ', 'TXT']
['STZ', 'UAL']
['STZ', 'VLO']
['STZ', 'VMC']
['STZ', 'XYL']
['SWK', 'SYMC']
['SWK', 'SYY']
['SWK', 'TAP']
['SWK', 'TDG']
['SWK', 'TGT']
['SWK', 'TIF']
['SWK', 'TMK']
['SWK', 'TPR']
['SWK', 'TROW']
['SWK', 'TRV']
['SWK', 'UAL']
['SWK', 'UDR']
['SWK', 'UNM']
['SWK', 'V']
['SWK', 'VFC']
['SWK', 'VIAB']
['SWK', 'VLO']
['SWK', 'VNO']
['SWK', 'VRSK']
['SWK', 'VRTX']
['SWK', 'VTR']
['SWK', 'WAT']
['SWK', 'WDC']
['SWK', 'WEC']
['SWK', 'WM']
['SWK', 'WU']
['SWK', 'WY']
['SWK', 'XEL']
['SWK', 'XL']
['SWK', 'XLNX']
['SWK', 'XRAY']
['SWK', 'XRX']
['SWK', 'XYL']
['SWK', 'YUM']
['SWK', 'ZION']
['SWK', 'ZTS']
['SYMC', 'TDG']
['SYM

In [8]:

listOfPosResult = pos_result.split('\n\n')
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

NWL SCG 2.51
5438
11996
Percentage of profiting pairs in Cluster: 45.331777259086365%


In [9]:
listRet.sort()
print(listRet)


[3.0082713401924365e-05, 6.858945717226561e-05, 0.00010600849845204152, 0.00016150536561994498, 0.00016712356761928505, 0.00016789036372188092, 0.00017807005721863335, 0.0001873172458864758, 0.0002154085871910638, 0.00022795108535547114, 0.00025038365877477986, 0.00025919735854159856, 0.000312331360104281, 0.0005399923645074977, 0.00057852614684478, 0.0005946175185371771, 0.0006516909758347822, 0.0007288639973961608, 0.0007475693628811086, 0.0007532625250524916, 0.0007910734130746278, 0.000803629573676261, 0.0008705470817096739, 0.000878147042015911, 0.0009177453403886205, 0.0009654782632601666, 0.0009762420341147049, 0.0009992137194464589, 0.0010413550358921954, 0.0010742337323318906, 0.001078111432806139, 0.0011469545138265858, 0.0012127694872799567, 0.0013375940239045293, 0.0014831911005965992, 0.0015612122142022145, 0.0016119827167800338, 0.0016185335526265022, 0.001668830439348028, 0.0017749871464339151, 0.0017832269358261498, 0.0018852778553410587, 0.002004697702782465, 0.0021383